In [3]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np
import sqlite3
from src.cmesrc.config import CMESRCV2_DB

In [4]:
conn = sqlite3.connect(CMESRCV2_DB)
conn.execute("PRAGMA foreign_keys = on")
cur = conn.cursor()

In [13]:
# First we need to know how many harps there are of each verification level

scores = pd.read_sql("""
            WITH ACCEPTED_HARPS AS (
                SELECT DISTINCT harpnum FROM HOURLY_PIXEL_BBOX
            )
            SELECT AH.harpnum, MIN(FCHA.verification_score) AS highest_verification_score FROM ACCEPTED_HARPS AH
            LEFT JOIN FINAL_CME_HARP_ASSOCIATIONS FCHA
            ON FCHA.harpnum = AH.harpnum
            GROUP BY AH.harpnum
            """, conn)
scores.replace(np.nan, 0, inplace=True)
scores["highest_verification_score"].value_counts().sort_index()

0.0    3088
1.0      68
2.0      66
3.0      51
4.0     178
5.0     108
Name: highest_verification_score, dtype: int64

In [19]:
# So the number of HARPS by split will be approx

n_splits = 5

split_counts = (scores["highest_verification_score"].value_counts().sort_index() / n_splits).astype(int).to_dict()
(scores["highest_verification_score"].value_counts().sort_index() / n_splits).astype(int).to_dict()

{0.0: 617, 1.0: 13, 2.0: 13, 3.0: 10, 4.0: 35, 5.0: 21}

In [20]:
# Now this is our dict that holds how many harps there are in each split-verif combination

assignment_counts = {
    i: {
        j: 0
        for j in split_counts.keys()
    } for i in range(n_splits)
}

assignment_counts

{0: {0.0: 0, 1.0: 0, 2.0: 0, 3.0: 0, 4.0: 0, 5.0: 0},
 1: {0.0: 0, 1.0: 0, 2.0: 0, 3.0: 0, 4.0: 0, 5.0: 0},
 2: {0.0: 0, 1.0: 0, 2.0: 0, 3.0: 0, 4.0: 0, 5.0: 0},
 3: {0.0: 0, 1.0: 0, 2.0: 0, 3.0: 0, 4.0: 0, 5.0: 0},
 4: {0.0: 0, 1.0: 0, 2.0: 0, 3.0: 0, 4.0: 0, 5.0: 0}}

In [ ]:
# Now we need a list of harps with overlaps

harps_with_overlaps = pd.read_sql("""
                                SELECT * FROM H
""")